In [1]:
import torch
from torch.utils import data
import numpy
import pandas
import cv2
import torchvision.transforms as transforms
from datasets import datasets
import torch.nn as nn
torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
datas=datasets.RNet_dataset()
loader=data.DataLoader(datas,batch_size=1,shuffle=False,num_workers=1)

In [3]:
from model import RNet
net=RNet.RNet()
device=torch.device('cuda:0')
net=net.to(device)

In [4]:
for m in net.modules():
    if isinstance(m,nn.Conv2d):
        nn.init.normal_(m.weight)
        nn.init.xavier_normal_(m.weight)
        nn.init.kaiming_normal_(m.weight)
        nn.init.constant_(m.bias,0)
    elif isinstance(m,nn.Linear):
        nn.init.normal_(m.weight)

In [5]:
import torch.optim as optim
criterion1=nn.CrossEntropyLoss()
criterion2=nn.MSELoss()
optimizer=optim.Adam(net.parameters(),lr=0.001,betas=(0.9,0.99))

In [8]:
beta1=[1,1,0]
beta2=[0,1,1]
for epoch in range(2):
    running_loss=0.0
    for i,data in enumerate(loader):
        img,det,label= data
        #img=img.type(torch.DoubleTensor)
        img,det,label = img.to(device),det.to(device),label.to(device)
        optimizer.zero_grad()
        outputs1,outputs2,_=net(img)
        outputs1.squeeze_(3),outputs2.squeeze_(3)
        outputs1.squeeze_(2),outputs2.squeeze_(2)
        det.squeeze_(1)
        if det==2:
            loss=criterion2(outputs2,label)*0.5
        elif det==0:
            loss=criterion1(outputs1,det)
        elif det==1:
            loss=criterion1(outputs1,det)+criterion2(outputs2,label)*0.5
        #if loss.item()>5:
         #   continue
        #loss=loss.type(torch.DoubleTensor)
        #loss=torch.DoubleTensor(loss)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        if i%2000==1999:
            print('[%d,%5d] loss:%.3f' %(epoch+1,i+1,running_loss/2000))
            running_loss=0.0
print('training finished')

RuntimeError: shape '[1, 576]' is invalid for input of size 371840